# Streamsight basic demo

Streamsight is a toolkit for recommendation systems. It is designed to provide
a comprehensive set of tools for the entire lifecycle of a recommendation
system, from the loading of dataset to the evaluation of the algorithms.

To install the toolkit, run `pip install streamsight`

The framework shown below will be covered in this demo to showcase to the
programmer how to utilise this toolkit.

![framework](../docs/source/_static/architecture.png)

## Data Load

The first segment of this demo covers the data load process for the demo. We will
cover the choice of dataset that the user can choose from first. The selection
of the dataset includes various dataset from Amazon and Yelp. The programmer
can choose his desired choice and instantiate an instance of the dataset as
shown below.

Furthermore, preprocessing filters such as `MinItemPerUser` is provided to the
programmer to filter the dataset on. There will be a default preprocessing
step that will be applied to all dataset - internal user and item id will be
created and replaces the original id. Please read the documentation for more
information on this in the preprocessing class.

To load the dataset, the programmer will invoke the method `load` to load the
dataset into memory. Please note that some datasets are huge and will take
some time to download.

In [ ]:
from streamsightv2.datasets import AmazonMusicDataset
from streamsightv2.preprocessing import MinItemsPerUser

dataset = AmazonMusicDataset()
dataset.add_filter(
    MinItemsPerUser(5, AmazonMusicDataset.ITEM_IX, AmazonMusicDataset.USER_IX)
)
data = dataset.load()

INFO     - streamsight.datasets.base - AmazonMusicDataset is loading dataset...
INFO     - streamsight.datasets.base - AmazonMusicDataset dataset loaded - Took 1.73s


The next step will be the splitting of the data. For the traditional split (single time point)
that is a single timestamp is chosen as the time value to split the data into train and test sets,
there will only be one split. For the sliding temporal split discussed in the paper which is being
implemented, there will be multiple splits, each split will have a different time value to split the
data into train and test sets.

For the purposes of `streamsight` the traditional terminology of train and test sets is replaced with
background and ground truth. As for the prediction to be made, we call the set unlabeled data which
the algorithm must predict on.

In [ ]:
from streamsightv2.settings import SingleTimePointSetting

setting = SingleTimePointSetting(
    background_t=1406851200,
    t_upper=1398556800,
    n_seq_data=1
)
# once a setting is defined, it can be used to split data
# the data will be stored in the attribute of the setting object
setting.split(data)

INFO     - streamsight.settings.single_time_point_setting - Splitting data at time 1406851200 with t_upper interval 1398556800


In [ ]:
from streamsightv2.settings import SlidingWindowSetting

setting_window = SlidingWindowSetting(
    background_t=1406851200,
    window_size=60 * 60 * 24 * 400, # 100 days
    n_seq_data=1,
)
setting_window.split(data)

4it [00:00, 36.98it/s]               

INFO     - streamsight.settings.sliding_window_setting - Finished split with window size 34560000 seconds. Number of splits: 4 in total.


## RecSys training

Training the RecSys algorithm is as straight forward. The choice of the algorithm
is selected by instantiating the class of algorithm choice then training the
model with the dataset from the setting. The setting class provides multiple
public function calls that can be used by the programmer.

As the algorithm that is designed in this package is a basic algorithm, some
extra processing step must be done on our end to treat the shape of the dataset
for the algorithm such that it is able to run successfully.

Note that the treating of the dataset is an optional parameter that the
programmer can choose not to apply in this platform. This will allow flexibility
on the programmer's end to evaluate the algorithm's capability.

We will demo a simple example below.

In [ ]:
############# Single global timeline split #############
from streamsightv2.algorithms import ItemKNNStatic

algo = ItemKNNStatic(K=10)

# Note that the data feed to the model must first be masked before
# it is fed to the model. The rational for this is to define the set
# of known user/item base knowledge such that the evaluation is
# well defined.
setting.background_data.mask_shape()
# each algorithm has a fit method that takes the training data and fits the model
algo.fit(setting.background_data)

setting.unlabeled_data.mask_shape(setting.background_data.shape, True, True)
X_pred = algo.predict(setting.unlabeled_data)

## Evaluation

The evaluation is done by comparing the predicted values with the true values. Below
we show a simple example of how to evaluate the prediction manually. The specific
class of metric will be instantiated and then used to calculate the score.

The above demo is to showcase how the pipeline and streamer work under the hood.
The recommended way to train and evaluate the algorithm on is to either
use the class `EvaluatorPipeline` or `EvaluatorStreamer` which are designed to
abstract the process and to allow the programmer to specify the parameters
needed to train the algorithm.

In [ ]:
from streamsightv2.metrics import PrecisionK

# Here we mask the ground truth data to match the shape of the prediction
# data. By dropping unknown users and items, we are only evaluating the
# users and items that are only known to the model.
setting.ground_truth_data.mask_shape(setting.background_data.shape,
                                     drop_unknown_user=True,
                                     drop_unknown_item=True)

metric = PrecisionK(10)
metric.calculate(setting.ground_truth_data.binary_values, X_pred)
print("The macro result of the algorithm is: {metric.macro_result}")

The macro result of the algorithm is: {metric.macro_result}


### Evaluation for sliding window setting

The evaluation for the sliding window setting case is a lot more complex and
uses an array of `InterationMatrix` unlike the single time point setting. For the
purposes of this demo, we will skip the code for the sliding window as it is the
same as the one above.

The difference is that the unlabeled_data and grouth_truth_data are now a list.
To get the the specific element, a simple list indexing would do. An example is
shown below.

In [6]:
setting_window.unlabeled_data[1]._df

,interactionid,uid,iid,ts
113,113,47,77,957312000
392,392,148,192,994982400
508,508,42,272,1006905600
1336,1336,393,434,1072137600
1372,1372,403,84,1074556800
...,...,...,...,...
436442,436442,31515,-1,1475884800
436443,436443,12463,-1,1475884800
436444,436444,43872,-1,1475884800
436445,436445,44409,-1,1475884800


# Pipeline to streamline API usage

The pipeline built uses an `Evaluator` class to help us run the entire process of
instantiating the model, data feed and metrics for us. To build the `Evaluator`
the builder class must first be created as shown below. Following the creation,
the programmer will define the algorithm/model of interest followed by the setting
that was created earlier along with the metric of interest.

Note that multiple algorithms and metrics can be added to the builder, allowing
for evaluation of multiple algorithms and metrics over a single run.

In [ ]:
from streamsightv2.evaluators import EvaluatorPipelineBuilder

builder = EvaluatorPipelineBuilder(
                     ignore_unknown_user=True,
                     ignore_unknown_item=True
                     )
builder.add_setting(setting_window)
builder.set_metric_K(10)
builder.add_algorithm("ItemKNNIncremental", {"K": 10})
builder.add_metric("PrecisionK")
builder.add_metric("RecallK")
evaluator = builder.build()

evaluator.run()

INFO     - streamsight.evaluators.evaluator_pipeline - Phase 1: Preparing the evaluator...


  0%|          | 0/4 [00:00<?, ?it/s]

INFO     - streamsight.evaluators.evaluator_pipeline - Phase 2: Evaluating the algorithms...
INFO     - streamsight.evaluators.evaluator_pipeline - Phase 3: Releasing the data...


 25%|██▌       | 1/4 [00:01<00:03,  1.26s/it]

INFO     - streamsight.evaluators.evaluator_pipeline - Phase 2: Evaluating the algorithms...
INFO     - streamsight.evaluators.evaluator_pipeline - Phase 3: Releasing the data...


 50%|█████     | 2/4 [00:02<00:02,  1.44s/it]

INFO     - streamsight.evaluators.evaluator_pipeline - Phase 2: Evaluating the algorithms...
INFO     - streamsight.evaluators.evaluator_pipeline - Phase 3: Releasing the data...


 75%|███████▌  | 3/4 [00:04<00:01,  1.56s/it]

INFO     - streamsight.evaluators.evaluator_pipeline - Phase 2: Evaluating the algorithms...
INFO     - streamsight.evaluators.evaluator_pipeline - Phase 3: Releasing the data...


100%|██████████| 4/4 [00:06<00:00,  1.56s/it]


### Metric results

We define 4 level of metrics computation. The below diagram will showcase an
example of how the metric would be computed using the example of Precision@10.

![metric computation](../docs/source/_static/metric_definition.png)

A `MetricAccumulator` class is used to abstract the computation from the evaluator
class. More details can be found within the `streamsight.metrics` module.

In [8]:
evaluator.metric_results("user")

user_id score
Algorithm                Timestamp    Metric                     
ItemKNNIncremental(K=10) t=1406851200 PrecisionK_10      21   0.0
                                      PrecisionK_10      26   0.0
                                      PrecisionK_10      29   0.0
                                      PrecisionK_10      32   0.0
                                      PrecisionK_10      38   0.0
...                                                     ...   ...
                         t=1510531200 RecallK_10      32747   0.0
                                      RecallK_10       8172   0.0
                                      RecallK_10      16373   0.0
                                      RecallK_10      24568   0.0
                                      RecallK_10      40953   0.0

[62768 rows x 2 columns]

In [9]:
evaluator.metric_results("window")

window_score  num_user
Algorithm                Timestamp    Metric                               
ItemKNNIncremental(K=10) t=1406851200 PrecisionK_10      0.001234      8755
                                      RecallK_10         0.006431      8755
                         t=1441411200 PrecisionK_10      0.001245     10603
                                      RecallK_10         0.005835     10603
                         t=1475971200 PrecisionK_10      0.000872      8602
                                      RecallK_10         0.003912      8602
                         t=1510531200 PrecisionK_10      0.000876      3424
                                      RecallK_10         0.004065      3424

In [10]:
evaluator.metric_results("micro")

micro_score  num_user
Algorithm                Metric                             
ItemKNNIncremental(K=10) PrecisionK_10    0.001099     31384
                         RecallK_10       0.005281     31384

In [11]:
evaluator.metric_results("macro")

macro_score  num_window
Algorithm                Metric                                
ItemKNNIncremental(K=10) PrecisionK_10     0.001057           4
                         RecallK_10        0.005061           4